# SAM Demo - Manual Setup & Testing

This notebook allows you to run the SAM demo setup step-by-step for testing and debugging.

**Use this for:**
- Verifying setup works correctly
- Debugging specific steps
- Understanding the setup flow
- Testing after code changes

**Prerequisites:**
- ACCOUNTADMIN role
- Accept SNOWFLAKE_PUBLIC_DATA_FREE marketplace listing


## Step 1: Infrastructure Setup
Create database, schemas, role, warehouse, and Git integration


In [ ]:
-- Create role and grant permissions
CREATE ROLE IF NOT EXISTS SAM_DEMO_ROLE;

GRANT USAGE ON DATABASE SAM_DEMO TO ROLE SAM_DEMO_ROLE;
GRANT ALL PRIVILEGES ON ALL SCHEMAS IN DATABASE SAM_DEMO TO ROLE SAM_DEMO_ROLE;
GRANT ALL PRIVILEGES ON FUTURE SCHEMAS IN DATABASE SAM_DEMO TO ROLE SAM_DEMO_ROLE;
GRANT ALL PRIVILEGES ON ALL TABLES IN DATABASE SAM_DEMO TO ROLE SAM_DEMO_ROLE;
GRANT ALL PRIVILEGES ON FUTURE TABLES IN DATABASE SAM_DEMO TO ROLE SAM_DEMO_ROLE;
GRANT ALL PRIVILEGES ON ALL VIEWS IN DATABASE SAM_DEMO TO ROLE SAM_DEMO_ROLE;
GRANT ALL PRIVILEGES ON FUTURE VIEWS IN DATABASE SAM_DEMO TO ROLE SAM_DEMO_ROLE;
GRANT ALL PRIVILEGES ON ALL PROCEDURES IN DATABASE SAM_DEMO TO ROLE SAM_DEMO_ROLE;
GRANT ALL PRIVILEGES ON FUTURE PROCEDURES IN DATABASE SAM_DEMO TO ROLE SAM_DEMO_ROLE;

GRANT ROLE SAM_DEMO_ROLE TO ROLE ACCOUNTADMIN;
GRANT ROLE SAM_DEMO_ROLE TO ROLE SYSADMIN;

SELECT 'Role created and permissions granted' AS status;


In [ ]:
-- Create warehouse (3X-LARGE for setup, will scale down later)
CREATE WAREHOUSE IF NOT EXISTS SAM_DEMO_WH
    WAREHOUSE_SIZE = '3X-LARGE'
    AUTO_SUSPEND = 60
    AUTO_RESUME = TRUE
    INITIALLY_SUSPENDED = FALSE;

ALTER WAREHOUSE SAM_DEMO_WH SET WAREHOUSE_SIZE = '3X-LARGE';

GRANT USAGE, OPERATE, MODIFY ON WAREHOUSE SAM_DEMO_WH TO ROLE SAM_DEMO_ROLE;

SELECT 'Warehouse created (3X-LARGE for fast setup)' AS status;


In [ ]:
-- Grant marketplace access and Cortex permissions
GRANT IMPORTED PRIVILEGES ON DATABASE SNOWFLAKE_PUBLIC_DATA_FREE TO ROLE SAM_DEMO_ROLE;

CREATE SNOWFLAKE INTELLIGENCE IF NOT EXISTS SNOWFLAKE_INTELLIGENCE_OBJECT_DEFAULT;

GRANT CREATE SNOWFLAKE INTELLIGENCE ON ACCOUNT TO ROLE SAM_DEMO_ROLE;
GRANT USAGE ON SNOWFLAKE INTELLIGENCE SNOWFLAKE_INTELLIGENCE_OBJECT_DEFAULT TO ROLE SAM_DEMO_ROLE;
GRANT MODIFY ON SNOWFLAKE INTELLIGENCE SNOWFLAKE_INTELLIGENCE_OBJECT_DEFAULT TO ROLE SAM_DEMO_ROLE;
GRANT USAGE ON SNOWFLAKE INTELLIGENCE SNOWFLAKE_INTELLIGENCE_OBJECT_DEFAULT TO ROLE PUBLIC;

GRANT CREATE AGENT ON SCHEMA SAM_DEMO.AI TO ROLE SAM_DEMO_ROLE;
GRANT CREATE CORTEX SEARCH SERVICE ON SCHEMA SAM_DEMO.AI TO ROLE SAM_DEMO_ROLE;
GRANT CREATE SEMANTIC VIEW ON SCHEMA SAM_DEMO.AI TO ROLE SAM_DEMO_ROLE;
GRANT BIND SERVICE ENDPOINT ON ACCOUNT TO ROLE SAM_DEMO_ROLE;

SELECT 'Marketplace and Cortex permissions granted' AS status;


In [ ]:
-- Setup Git integration
-- ⚠️ IMPORTANT: Update USERNAME and PASSWORD with your GitHub credentials!

CREATE OR REPLACE SECRET SAM_DEMO.PUBLIC.GITHUB_SECRET
  TYPE = PASSWORD
  USERNAME = 'your_github_username'  -- CHANGE THIS!
  PASSWORD = 'your_github_pat'       -- CHANGE THIS!
  COMMENT = 'GitHub PAT for SAM demo';

GRANT USAGE, READ ON SECRET SAM_DEMO.PUBLIC.GITHUB_SECRET TO ROLE SAM_DEMO_ROLE;

CREATE OR REPLACE API INTEGRATION GITHUB_INTEGRATION_SAM_DEMO
  API_PROVIDER = git_https_api
  API_ALLOWED_PREFIXES = ('https://github.com/')
  ALLOWED_AUTHENTICATION_SECRETS = (SAM_DEMO.PUBLIC.GITHUB_SECRET)
  ENABLED = TRUE;

CREATE OR REPLACE GIT REPOSITORY SAM_DEMO.PUBLIC.sam_demo_repo
  API_INTEGRATION = GITHUB_INTEGRATION_SAM_DEMO
  GIT_CREDENTIALS = SAM_DEMO.PUBLIC.GITHUB_SECRET
  ORIGIN = 'https://github.com/sfc-gh-dshemsi/sfguide-agentic-ai-for-asset-management.git';

GRANT READ ON GIT REPOSITORY SAM_DEMO.PUBLIC.sam_demo_repo TO ROLE SAM_DEMO_ROLE;

ALTER GIT REPOSITORY SAM_DEMO.PUBLIC.sam_demo_repo FETCH;

SELECT 'Git integration configured and repo fetched' AS status;


## Step 2: Load Procedure Definitions
Load modular SQL files from Git to define stored procedures


In [ ]:
-- Switch to demo role and warehouse
USE ROLE SAM_DEMO_ROLE;
USE WAREHOUSE SAM_DEMO_WH;
USE DATABASE SAM_DEMO;

SELECT CURRENT_ROLE() AS role, 
       CURRENT_WAREHOUSE() AS warehouse, 
       CURRENT_DATABASE() AS database;


In [ ]:
-- Load data generation procedures from Git
EXECUTE IMMEDIATE FROM @SAM_DEMO.PUBLIC.sam_demo_repo/branches/main/scripts/data_generation.sql;

SELECT 'Data generation procedures loaded (SETUP_SAM_DEMO)' AS status;


In [ ]:
-- Load AI components procedures from Git
EXECUTE IMMEDIATE FROM @SAM_DEMO.PUBLIC.sam_demo_repo/branches/main/scripts/ai_components.sql;

SELECT 'AI component procedures loaded (SETUP_AI_COMPONENTS, SETUP_AGENTS)' AS status;


In [ ]:
-- Load custom tools from Git
EXECUTE IMMEDIATE FROM @SAM_DEMO.PUBLIC.sam_demo_repo/branches/main/scripts/custom_tools.sql;

SELECT 'Custom tools loaded (PDF generator, M&A simulation)' AS status;


In [ ]:
-- Verify procedures were created
SHOW PROCEDURES IN SCHEMA SAM_DEMO.PUBLIC;
SHOW PROCEDURES IN SCHEMA SAM_DEMO.AI;


## Step 3: Generate Data
Run the master data generation procedure

⏱️ **This takes ~10 minutes** - generates all dimensions, facts, market data, and documents


In [ ]:
-- Generate all data (dimensions, facts, market data, documents)
CALL SAM_DEMO.PUBLIC.SETUP_SAM_DEMO(FALSE);


In [ ]:
-- Verify tables were created
SELECT 'Dimension Tables' AS category, COUNT(*) AS count 
FROM INFORMATION_SCHEMA.TABLES 
WHERE TABLE_SCHEMA = 'CURATED' AND TABLE_NAME LIKE 'DIM_%'
UNION ALL
SELECT 'Fact Tables', COUNT(*) 
FROM INFORMATION_SCHEMA.TABLES 
WHERE TABLE_SCHEMA = 'CURATED' AND TABLE_NAME LIKE 'FACT_%'
UNION ALL
SELECT 'Document RAW Tables', COUNT(*) 
FROM INFORMATION_SCHEMA.TABLES 
WHERE TABLE_SCHEMA = 'RAW' AND TABLE_NAME LIKE '%_RAW'
UNION ALL
SELECT 'Document CORPUS Tables', COUNT(*) 
FROM INFORMATION_SCHEMA.TABLES 
WHERE TABLE_SCHEMA = 'CURATED' AND TABLE_NAME LIKE '%_CORPUS';


In [ ]:
-- Sample data check - verify rows were populated
SELECT 'DIM_SECURITY' AS table_name, COUNT(*) AS row_count 
FROM SAM_DEMO.CURATED.DIM_SECURITY
UNION ALL
SELECT 'DIM_PORTFOLIO', COUNT(*) 
FROM SAM_DEMO.CURATED.DIM_PORTFOLIO
UNION ALL
SELECT 'FACT_POSITION_DAILY_ABOR', COUNT(*) 
FROM SAM_DEMO.CURATED.FACT_POSITION_DAILY_ABOR
UNION ALL
SELECT 'BROKER_RESEARCH_CORPUS', COUNT(*) 
FROM SAM_DEMO.CURATED.BROKER_RESEARCH_CORPUS;


## Step 4: Build AI Components
Create semantic views and Cortex Search services

⏱️ **This takes ~2-3 minutes**


In [ ]:
-- Create semantic views and search services
CALL SAM_DEMO.PUBLIC.SETUP_AI_COMPONENTS();


In [ ]:
-- Verify semantic views
SHOW SEMANTIC VIEWS IN SCHEMA SAM_DEMO.AI;


In [ ]:
-- Verify search services
SHOW CORTEX SEARCH SERVICES IN SCHEMA SAM_DEMO.AI;


## Step 5: Create Agents
Create all 9 Cortex agents

⏱️ **This takes ~1 minute**


In [ ]:
-- Create all agents
CALL SAM_DEMO.PUBLIC.SETUP_AGENTS();


In [ ]:
-- Verify agents were created
SHOW AGENTS IN SCHEMA SAM_DEMO.AI;


## Step 6: Validation & Testing
Verify everything is working correctly


In [ ]:
-- Final component summary
SELECT '=== SETUP SUMMARY ===' AS summary, '' AS actual, '' AS expected
UNION ALL SELECT '────────────────────', '────────', '────────'
UNION ALL
SELECT 'Semantic Views', 
       (SELECT COUNT(*)::STRING FROM INFORMATION_SCHEMA.SEMANTIC_VIEWS WHERE SEMANTIC_VIEW_SCHEMA = 'AI'),
       '10'
UNION ALL
SELECT 'Search Services',
       (SELECT COUNT(*)::STRING FROM INFORMATION_SCHEMA.CORTEX_SEARCH_SERVICES WHERE SERVICE_SCHEMA = 'AI'),
       '16'
UNION ALL
SELECT 'Agents',
       (SELECT COUNT(*)::STRING FROM INFORMATION_SCHEMA.AGENTS WHERE AGENT_SCHEMA = 'AI'),
       '9'
UNION ALL
SELECT 'Dimension Tables',
       (SELECT COUNT(*)::STRING FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'CURATED' AND TABLE_NAME LIKE 'DIM_%'),
       '~10'
UNION ALL
SELECT 'Fact Tables',
       (SELECT COUNT(*)::STRING FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'CURATED' AND TABLE_NAME LIKE 'FACT_%'),
       '~15'
UNION ALL
SELECT 'Document Tables',
       (SELECT COUNT(*)::STRING FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_SCHEMA = 'RAW' AND TABLE_NAME LIKE '%_RAW'),
       '~15';


In [ ]:
-- Test semantic view - query some sample data
SELECT * FROM SAM_DEMO.AI.SAM_ANALYST_VIEW LIMIT 5;


In [ ]:
-- Test search service - search for Microsoft documents
SELECT SNOWFLAKE.CORTEX.SEARCH_PREVIEW(
    'SAM_DEMO.AI.SAM_BROKER_RESEARCH',
    '{
        "query": "Microsoft",
        "columns": ["DOCUMENT_TITLE", "DOCUMENT_TEXT"],
        "limit": 3
    }'
) AS search_results;


## Step 7: Cleanup & Optimization
Scale down warehouse for cost savings


In [ ]:
-- Scale down warehouse to MEDIUM and suspend
ALTER WAREHOUSE SAM_DEMO_WH SET WAREHOUSE_SIZE = 'MEDIUM';
ALTER WAREHOUSE SAM_DEMO_WH SUSPEND;

SELECT 'Warehouse scaled to MEDIUM and suspended - setup complete!' AS status;


## 🎉 Setup Complete!

**Next steps:**
1. Open Snowflake Intelligence UI
2. Select an agent (e.g., **Portfolio Copilot**)
3. Try demo prompts:
   - `What are my top 10 holdings in SAM Technology & Infrastructure?`
   - `Check for concentration breaches across all portfolios`
   - `What is the latest research on Microsoft?`
   - `I've received an alert that META has been downgraded to ESG grade CCC. Can you verify this breach for the SAM ESG Leaders Global Equity portfolio?`

**Components created:**
- ✅ 10 Semantic Views (Cortex Analyst)
- ✅ 16 Cortex Search Services
- ✅ 9 Cortex Agents
- ✅ 50+ dimension and fact tables
- ✅ 500+ synthetic documents
